In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.compose import make_column_selector as selector

In [2]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100
plt.rcParams["figure.figsize"] = (10,8)
plt.style.use('seaborn-darkgrid')

In [3]:
all_lead_acs_county_raw = pd.read_csv("../data/census/cutoff_all_lead_acs_county.csv", low_memory = False, index_col=0)

# Cleaning

## Missing Value

In [4]:
for col in all_lead_acs_county_raw.columns:
    if all_lead_acs_county_raw[col].isna().sum()/ all_lead_acs_county_raw.shape[0] > 0.8:
        print(col)

MASTER_RECORD_ID
SALUTATION
SUFFIX
TITLE
COUNTRY
LATITUDE
LONGITUDE
GEOCODE_ACCURACY
FAX
WEBSITE
DESCRIPTION
INDUSTRY
RATING
ANNUAL_REVENUE
NUMBER_OF_EMPLOYEES
LAST_VIEWED_DATE
LAST_REFERENCED_DATE
JIGSAW
JIGSAW_CONTACT_ID
EMAIL_BOUNCED_REASON
EMAIL_BOUNCED_DATE
INDIVIDUAL_ID
POSTAL_CODE_C
ALTERNATE_EMAIL_C
ALTERNATE_PHONE_C
TOTAL_SALES_C
BOOK_SIZE_SOW_PREFERRED_C
EVENT_TYPE_C
AVERAGE_LIST_PRICE_C
FSBO_ADDRESS_C
FSBO_LISTING_PRICE_C
FIRST_EVENT_DATE_C
SALES_LOFT_1_MOST_RECENT_CADENCE_NAME_C
SALES_LOFT_1_MOST_RECENT_CADENCE_NEXT_STEP_DUE_DATE_C
SALES_LOFT_1_MOST_RECENT_LAST_COMPLETED_STEP_C
DISQUALIFIED_NOTES_C
FA_LOYAL_LEAD_C
LS_SALES_C
PREFERRED_PROVIDER_BY_LISTING_COUNT_C
PRODUCT_DEMO_DATE_C
PREFERRED_ESCROW_COMPANY_C
APP_DOWNLOAD_DATE_C
PREFERRED_TITLE_COMPANY_C
ORDER_ENTRY_NOTES_C
LISTING_PRICE_C
TOTAL_MODUS_TRANSACTION_COUNT_C
PI_NEEDS_SCORE_SYNCED_C
PI_PARDOT_LAST_SCORED_AT_C
PI_CAMPAIGN_C
PI_COMMENTS_C
PI_CONVERSION_DATE_C
PI_CONVERSION_OBJECT_NAME_C
PI_CONVERSION_OBJECT_TYPE_C


In [5]:
# drop the columns with missing value over 80%
all_lead_acs_county = all_lead_acs_county_raw.loc[:, (all_lead_acs_county_raw.isnull().sum(axis=0) / all_lead_acs_county_raw.shape[0] < 0.8)]

## Inpute value & Drop Columns

### Numerical columns

In [6]:
numerical_columns_selector = selector(dtype_include='number')
numerical_columns = numerical_columns_selector(all_lead_acs_county)
numerical_columns

['index',
 'TOTAL_TRANSACTION_COUNT_C',
 'TOTAL_LISTING_COUNT_C',
 'AVERAGE_CLOSE_PRICE_C',
 'BS_COUNT_C',
 'LEAD_NUMBER_C',
 'MATRIX_UNIQUE_ID_C',
 'PMXAGENT_ID_C',
 'Percent!!HOUSEHOLDS BY TYPE!!Total households',
 'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household',
 'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household',
 'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present',
 'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone',
 'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone!!65 years and over',
 'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present',
 'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present!!Householder living alone',
 'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Fema

In [7]:
for col in numerical_columns:
    print(round(all_lead_acs_county[col].isnull().sum()/ all_lead_acs_county.shape[0],4), ":",col)

0.0 : index
0.7081 : TOTAL_TRANSACTION_COUNT_C
0.3691 : TOTAL_LISTING_COUNT_C
0.4496 : AVERAGE_CLOSE_PRICE_C
0.6303 : BS_COUNT_C
0.225 : LEAD_NUMBER_C
0.7851 : MATRIX_UNIQUE_ID_C
0.3999 : PMXAGENT_ID_C
0.0758 : Percent!!HOUSEHOLDS BY TYPE!!Total households
0.0758 : Percent!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household
0.0758 : Percent!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household
0.0758 : Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present
0.0758 : Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone
0.0758 : Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone!!65 years and over
0.0758 : Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present
0.0758 : Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present!!Ho

In [8]:
numeric_drop_cols = ['index',
 'TOTAL_TRANSACTION_COUNT_C',
 'TOTAL_LISTING_COUNT_C',
 'AVERAGE_CLOSE_PRICE_C',
 'BS_COUNT_C',
 'LEAD_NUMBER_C',
 'MATRIX_UNIQUE_ID_C',
 'PMXAGENT_ID_C',]

In [9]:
all_lead_acs_county = all_lead_acs_county.drop(numeric_drop_cols, axis =1)

In [10]:
# use state level average to impute missing value of percentage columns

In [11]:
def impute_group_mean(df:pd.DataFrame, cols:list, group_col:str) -> pd.DataFrame:

    for col in cols:
        df.loc[:,col] = df[col].fillna(df.groupby(group_col)[col].transform("mean"))
    return df

In [12]:
# transform the full state name to acronym and for imputing missing value
# using state value to impute first, if no state information, use nation value

In [13]:
state_acs = pd.read_csv("../data/census/state_acs.csv")
nation_acs = pd.read_csv("../data/census/nation_acs.csv")

In [14]:
def acs_col_impute(df:pd.DataFrame, state_df:pd.DataFrame, nation_df:pd.DataFrame) -> pd.DataFrame:
    # state imputation
    for state in df["STATE_STD"].unique():
        state_fillna_dict = {k: v for d in state_df[[i for i in state_df.columns if i.startswith("Percent!!")]][state_df["STATE_STD"] == state].to_dict(orient="records") for k, v in d.items()}
        df[df["STATE_STD"] == state] = df[df["STATE_STD"] == state].fillna(state_fillna_dict)

    # nation imputation
    nation_fillna_dict = {k: v for d in nation_df[[i for i in nation_df.columns if i.startswith("Percent!!")]].to_dict(orient="records") for k, v in d.items()}
    df = df.fillna(nation_fillna_dict)

    return df

In [15]:
all_lead_acs_county = acs_col_impute(all_lead_acs_county,state_acs, nation_acs)

In [16]:
#### drop the acs value columns, since national value and state value are imputed
percent_col = [col for col in all_lead_acs_county.columns if (col.startswith("Percent!!"))]
value_percent_col = [col for col in percent_col if ((all_lead_acs_county[col].values) >=100).any()]

In [17]:
all_lead_acs_county = all_lead_acs_county.drop(value_percent_col, axis =1)

### Categorical Columns

In [18]:
# notes
# "RATING" may not be the column for ratings, check out other columns
# "MOST_RECENT_CAMPAIGN_C" should be modified by time
# "LEVEL_VETTING_C","LEAD_RANKING_C" both have level information, but "LEAD_RANKING_C" doesn't contain "level 2" "level 1"
# exclude_col = ["RATING","LEAD_RANKING_C","MOST_RECENT_CAMPAIGN_C"]
# ord_col = ["LEVEL_VETTING_C","LEAD_RANKING_C"]
# cat_col = ["RECORD_TYPE_ID","STATE","COUNTRY","LEAD_SOURCE","STATUS","RATING", "CREATED_BY_ID", "LAST_MODIFIED_BY_ID", "EMAIL_BOUNCED_REASON",
#              "MOST_RECENT_CAMPAIGN_C","EVENT_TYPE_C","COUNTY_C","FA_LOYAL_LEAD_C", "PREFERRED_ESCROW_COMPANY_C", "ROLE_C","AGENT_ROLE_C","WEB_LEAD_C",
#                ]

In [19]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(all_lead_acs_county)
categorical_columns

['ID',
 'LAST_NAME',
 'FIRST_NAME',
 'MIDDLE_NAME',
 'NAME',
 'RECORD_TYPE_ID',
 'COMPANY',
 'STREET',
 'CITY',
 'STATE',
 'POSTAL_CODE',
 'PHONE',
 'MOBILE_PHONE',
 'EMAIL',
 'PHOTO_URL',
 'LEAD_SOURCE',
 'STATUS',
 'OWNER_ID',
 'CONVERTED_DATE',
 'CONVERTED_ACCOUNT_ID',
 'CONVERTED_CONTACT_ID',
 'CONVERTED_OPPORTUNITY_ID',
 'CREATED_DATE',
 'CREATED_BY_ID',
 'LAST_MODIFIED_DATE',
 'LAST_MODIFIED_BY_ID',
 'SYSTEM_MODSTAMP',
 'LAST_ACTIVITY_DATE',
 'LAST_TRANSFER_DATE',
 'MOST_RECENT_CAMPAIGN_C',
 'FIRST_TASK_DATE_C',
 'BROKERAGE_1_C',
 'COUNTY_C',
 'ROLE_C',
 'AGENT_MLS_ID_C',
 'LEVEL_VETTING_C',
 'LICENSE_NO_C',
 'AGENT_ROLE_C',
 'LAST_MODIFIED_C',
 'DISQUALIFIED_REASONS_C',
 '_FIVETRAN_SYNCED',
 'EMAIL_UNKNOWN_DELIVERABILITY_C',
 'WEB_LEAD_C',
 'KNOCK_WEBHOOK_2021_C',
 'DO_NOT_CONTACT_C',
 'MOBILE_INDEXED_C',
 'STATE_STD',
 'COUNTY_FULL',
 'Geographic Area Name']

In [20]:
for col in all_lead_acs_county.select_dtypes("O").columns:
    if all_lead_acs_county[col].nunique() <100:
        print(all_lead_acs_county[col].nunique(), ":", col)

7 : RECORD_TYPE_ID
23 : STATE
34 : LEAD_SOURCE
6 : STATUS
49 : OWNER_ID
46 : CREATED_BY_ID
53 : LAST_MODIFIED_BY_ID
22 : MOST_RECENT_CAMPAIGN_C
17 : COUNTY_C
11 : ROLE_C
5 : LEVEL_VETTING_C
16 : AGENT_ROLE_C
9 : DISQUALIFIED_REASONS_C
2 : EMAIL_UNKNOWN_DELIVERABILITY_C
2 : WEB_LEAD_C
2 : KNOCK_WEBHOOK_2021_C
2 : DO_NOT_CONTACT_C
19 : STATE_STD


In [21]:
# for col in all_lead_acs_county.select_dtypes("O").columns:
#     if all_lead_acs_county[col].nunique() <100:
#         display(all_lead_acs_county[col].value_counts(normalize= True))

In [22]:
categorical_drop_cols_nunique = []
for col in all_lead_acs_county.select_dtypes("O").columns:
    if all_lead_acs_county[col].nunique() > 100:
        categorical_drop_cols_nunique.append(col)

categorical_drop_cols_nunique
    

['ID',
 'LAST_NAME',
 'FIRST_NAME',
 'MIDDLE_NAME',
 'NAME',
 'COMPANY',
 'STREET',
 'CITY',
 'POSTAL_CODE',
 'PHONE',
 'MOBILE_PHONE',
 'EMAIL',
 'PHOTO_URL',
 'CONVERTED_DATE',
 'CONVERTED_ACCOUNT_ID',
 'CONVERTED_CONTACT_ID',
 'CONVERTED_OPPORTUNITY_ID',
 'CREATED_DATE',
 'LAST_MODIFIED_DATE',
 'SYSTEM_MODSTAMP',
 'LAST_ACTIVITY_DATE',
 'LAST_TRANSFER_DATE',
 'FIRST_TASK_DATE_C',
 'BROKERAGE_1_C',
 'AGENT_MLS_ID_C',
 'LICENSE_NO_C',
 'LAST_MODIFIED_C',
 '_FIVETRAN_SYNCED',
 'MOBILE_INDEXED_C',
 'COUNTY_FULL',
 'Geographic Area Name']

In [23]:
categorical_drop_cols_nunique.remove("ID")

In [24]:
all_lead_acs_county = all_lead_acs_county.drop(categorical_drop_cols_nunique, axis = 1)

In [25]:
categorical_drop_cols_manual = ["RECORD_TYPE_ID", "STATE", "OWNER_ID","CREATED_BY_ID","LAST_MODIFIED_BY_ID", "MOST_RECENT_CAMPAIGN_C",
"COUNTY_C", "DISQUALIFIED_REASONS_C", "EMAIL_UNKNOWN_DELIVERABILITY_C", "WEB_LEAD_C", "KNOCK_WEBHOOK_2021_C","DO_NOT_CONTACT_C"]

In [26]:
all_lead_acs_county = all_lead_acs_county.drop(categorical_drop_cols_manual, axis = 1)

### Impute Missing Value

In [27]:
cat_keep_list = ["LEVEL_VETTING_C"]
cat_fillna_list = [col for col in all_lead_acs_county.select_dtypes("O").columns if col not in cat_keep_list]
cat_fillna_list

['ID', 'LEAD_SOURCE', 'STATUS', 'ROLE_C', 'AGENT_ROLE_C', 'STATE_STD']

In [28]:
for col in cat_fillna_list:
    all_lead_acs_county[col] = all_lead_acs_county[col].fillna("UNK")

### Boolean Columns 

In [29]:
[col for col in all_lead_acs_county.select_dtypes(bool).columns]

['IS_DELETED',
 'HAS_OPTED_OUT_OF_EMAIL',
 'IS_CONVERTED',
 'IS_UNREAD_BY_OWNER',
 'DO_NOT_CALL',
 'HAS_OPTED_OUT_OF_FAX',
 'APEX_PROCESSED_C',
 'KNOCK_WEBHOOK_C',
 'EMAIL_BOUNCED_C',
 'IS_LEAD_C',
 'PARDOT_REASSIGNMENT_C',
 'DIGITAL_TE_USER_C',
 'CHECKBOX_C',
 'KCA_C']

In [30]:
for col in all_lead_acs_county.select_dtypes(bool).columns:
    display(all_lead_acs_county[col].value_counts(normalize = True))

False    1.0
Name: IS_DELETED, dtype: float64

False    0.983239
True     0.016761
Name: HAS_OPTED_OUT_OF_EMAIL, dtype: float64

False    0.554601
True     0.445399
Name: IS_CONVERTED, dtype: float64

False    0.687567
True     0.312433
Name: IS_UNREAD_BY_OWNER, dtype: float64

False    0.990756
True     0.009244
Name: DO_NOT_CALL, dtype: float64

...

True     0.783136
False    0.216864
Name: IS_LEAD_C, dtype: float64

False    1.0
Name: PARDOT_REASSIGNMENT_C, dtype: float64

False    0.989215
True     0.010785
Name: DIGITAL_TE_USER_C, dtype: float64

False    0.995751
True     0.004249
Name: CHECKBOX_C, dtype: float64

False    0.983753
True     0.016247
Name: KCA_C, dtype: float64

In [31]:
boolean_drop_cols_manual = ["IS_DELETED","IS_UNREAD_BY_OWNER","DO_NOT_CALL","HAS_OPTED_OUT_OF_FAX","APEX_PROCESSED_C",
 "KNOCK_WEBHOOK_C", "EMAIL_BOUNCED_C","IS_LEAD_C","PARDOT_REASSIGNMENT_C","DIGITAL_TE_USER_C","CHECKBOX_C","KCA_C"]

In [32]:
all_lead_acs_county = all_lead_acs_county.drop(boolean_drop_cols_manual, axis = 1)

# Encoding

In [33]:
#### feature engineering from DataRobot


# encoding

# Categorical: ordinal encoding
    # For a categorical feature, convert categorical levels to an ordinal scale. The ordinal scale is 0 to (unique values of categorical_var) - 1. 
    # Rare categories (=other) and missing values are encoded as -1 and -2, respectively. 
    # Mapping is based on the lexicographic ordering of the categorical values, the frequency of the levels, the response, or is done randomly.     

# Missing value inputation: replace "NA" with "UNK"


# Numeric: Impute missing values (with -9999)
    # A numeric feature is imputed with the arbitrary value (default: -9999) if it:
        # has enough rows with finite values (e.g., > t, default: 10).
        # has large number of rows with NaN (e.g., > min_cna, default: 5).
        # is not configured as a monotonic-constrained feature.

## Categorical

In [34]:
select_col = []
for col in all_lead_acs_county.select_dtypes("O").columns:
    if all_lead_acs_county[col].nunique() < 100:
        select_col.append(col)
        print(col, ":",all_lead_acs_county[col].nunique())

LEAD_SOURCE : 35
STATUS : 6
ROLE_C : 12
LEVEL_VETTING_C : 5
AGENT_ROLE_C : 17
STATE_STD : 20


In [35]:
print(select_col)

['LEAD_SOURCE', 'STATUS', 'ROLE_C', 'LEVEL_VETTING_C', 'AGENT_ROLE_C', 'STATE_STD']


In [36]:
ord_col = ["LEVEL_VETTING_C"]
cat_col = ["LEAD_SOURCE","STATUS","ROLE_C","AGENT_ROLE_C","STATE_STD"]

In [37]:
all_lead_acs_county[ord_col+ cat_col].isna().sum()

LEVEL_VETTING_C    5344
LEAD_SOURCE           0
STATUS                0
ROLE_C                0
AGENT_ROLE_C          0
STATE_STD             0
dtype: int64

### Categorical level condense

In [38]:
def condense_category(col:pd.Series, min_freq=0.01, new_name='Other')-> pd.Series:
# function for shrink the level based on frequency
# do the value count with normalization
# set the cutoff threshold
    series = pd.value_counts(col, normalize= True)
    mask = (series).lt(min_freq)
    return pd.Series(np.where(col.isin(series[mask].index), new_name, col))

In [39]:
for col in cat_col:
    all_lead_acs_county[col] = condense_category(all_lead_acs_county[col])

###  Ordinal Encoding

In [40]:
def ordinal_encoding(df:pd.DataFrame, col_list:list) -> pd.DataFrame:
    for col in col_list:
        ord_enc = OrdinalEncoder()
        df[col +"_ORD"] = df[col].apply(ord_enc.fit_transform)
    return df

In [41]:
all_lead_acs_county["LEVEL_VETTING_C"]= all_lead_acs_county["LEVEL_VETTING_C"].fillna(np.nan)

In [42]:
level_mapping = ['Level 3 - Less than 4 listings/yr', 'Level 2 - 4+ listings/yr', "Level 1 - 10+ listings/yr",  'Level Gold 20-29 Listings/yr', 'Level Platinum 30+ Listings/yr']

In [43]:
encoder = OrdinalEncoder(categories=[level_mapping], handle_unknown = "use_encoded_value", unknown_value= np.nan, encoded_missing_value= -1)

In [44]:
encoder.fit(all_lead_acs_county[["LEVEL_VETTING_C"]])

OrdinalEncoder(categories=[['Level 3 - Less than 4 listings/yr',
                            'Level 2 - 4+ listings/yr',
                            'Level 1 - 10+ listings/yr',
                            'Level Gold 20-29 Listings/yr',
                            'Level Platinum 30+ Listings/yr']],
               encoded_missing_value=-1, handle_unknown='use_encoded_value',
               unknown_value=nan)

In [45]:
all_lead_acs_county["LEVEL_VETTING_C"]= encoder.transform(all_lead_acs_county[["LEVEL_VETTING_C"]])

In [46]:
# One Hot Encoding

In [47]:
all_lead_acs_county = pd.get_dummies(all_lead_acs_county, columns= cat_col)

In [48]:
### Label Encoder
# https://inria.github.io/scikit-learn-mooc/python_scripts/03_categorical_pipeline.html

In [49]:
# target encoding
# te = ce.TargetEncoder(cols = "STATE_STD")

In [50]:
# te.fit_transform(all_lead_acs_county["STATE_STD"], all_lead_acs_county["IS_CONVERTED"]).value_counts()

In [51]:
# all_lead_acs_county["STATE_STD"].value_counts()

## Numerical

In [52]:
all_lead_acs_county.head()

,ID,HAS_OPTED_OUT_OF_EMAIL,IS_CONVERTED,LEVEL_VETTING_C,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household,"Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone!!65 years and over","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present!!Householder living alone","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present!!Householder living alone!!65 years and over",Percent!!HOUSEHOLDS BY TYPE!!Total households!!Households with one or more people under 18 years,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Households with one or more people 65 years and over,Percent!!RELATIONSHIP!!Population in households!!Householder,Percent!!RELATIONSHIP!!Population in households!!Spouse,Percent!!RELATIONSHIP!!Population in households!!Unmarried partner,Percent!!RELATIONSHIP!!Population in households!!Child,Percent!!RELATIONSHIP!!Population in households!!Other relatives,Percent!!RELATIONSHIP!!Population in households!!Other nonrelatives,Percent!!MARITAL STATUS!!Males 15 years and over!!Never married,"Percent!!MARITAL STATUS!!Males 15 years and over!!Now married, except separated",Percent!!MARITAL STATUS!!Males 15 years and over!!Separated,Percent!!MARITAL STATUS!!Males 15 years and over!!Widowed,Percent!!MARITAL STATUS!!Males 15 years and over!!Divorced,Percent!!MARITAL STATUS!!Females 15 years and over!!Never married,"Percent!!MARITAL STATUS!!Females 15 years and over!!Now married, except separated",Percent!!MARITAL STATUS!!Females 15 years and over!!Separated,Percent!!MARITAL STATUS!!Females 15 years and over!!Widowed,Percent!!MARITAL STATUS!!Females 15 years and over!!Divorced,"Percent!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Unmarried women (widowed, divorced, and never married)",Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!Less than 1 year,Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!1 or 2 years,Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!3 or 4 years,Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!5 or more years,Percent!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years!!Who are female,"Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school, preschool",Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,Percent!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Percent!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",Percent!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),"Percent!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree",Percent!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Percent!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bache

In [53]:
all_lead_acs_county.to_csv("../data/model/all_lead_acs_model.csv",index = None)